In [3]:
from modAL.models import ActiveLearner
from modAL.models import Committee
from modAL.uncertainty import entropy_sampling, uncertainty_sampling, margin_sampling
from modAL.disagreement import vote_entropy
from modAL.disagreement import KL_max_disagreement
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import datasets
import numpy as np
import pandas as pd

/home/rishav/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/rishav/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/rishav/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Problem 1(Using pool based sampling here)
## This section also has all experiments related to pool based pooling

## Method for uncertainity sampling

In [21]:
#function for 1(a), this uses pool based pooling
def active_learning_single(X,y,n_queries,n_keep,query_strategy):
    idx_initial = np.random.choice(range(len(X)), size=n_keep, replace=False)
    idx_rem = [i for i in range(len(X)) if i not in idx_initial]

    x_initial = X[idx_initial]
    y_initial = y[idx_initial]

    x_rem = X[idx_rem]
    y_rem = y[idx_rem]
    
    act_learner = ActiveLearner(estimator=LogisticRegression(),X_training=x_initial,y_training=y_initial,query_strategy=query_strategy)
    performance_track = []
    performance_track.append(act_learner.score(x_rem,y_rem))
    
    for i in range(n_queries):
        #picking out next 10% of data
        idx_next_10 = np.random.choice(range(x_rem.shape[0]),size=int(x.shape[0]*.1),replace=False)
        x_rem_10 = x_rem[idx_next_10]
        y_rem_10 = y_rem[idx_next_10]
        
        for j in range(x_rem_10.shape[0]):
            query_idx, query_instance = act_learner.query(x_rem_10)
            #y_pred = act_learner.predict(x_rem)
            act_learner.teach(x_rem_10[query_idx], y_rem_10[query_idx])
            #print(accuracy_score(y_rem, y_pred))
            x_rem_10 = np.delete(x_rem_10, query_idx, 0)
            y_rem_10 = np.delete(y_rem_10, query_idx, 0)
    
        x_rem = np.delete(x_rem, idx_next_10, 0) 
        y_rem = np.delete(y_rem, idx_next_10, 0)
        performance_track.append(act_learner.score(x_rem,y_rem))
    return performance_track

## Dataset Loading & Preprocessing

In [27]:
data = pd.read_csv('Data/pendigits.tra').to_numpy()
x = data[:,:16]
y = data[:,16]

## Method for QBC

In [112]:
def active_learning_committee(X,y,n_queries,n_committee,n_keep,query_strategy):
    learners = []
    x_initial = X[idx_initial]
    y_initial = y[idx_initial]

    x_rem = X[idx_rem]
    y_rem = y[idx_rem]
    for i in range(n_committee):
        idx_initial = np.random.choice(range(len(X)), size=n_keep, replace=False)
        idx_rem = [i for i in range(len(X)) if i not in idx_initial]

        learner = ActiveLearner(estimator=LogisticRegression(),X_training=x_initial,y_training=y_initial)
        learners.append(learner)
    
    committee = Committee(learner_list=learners,query_strategy=query_strategy)
    comm_perform = []

    comm_perform.append(committee.score(x_rem,y_rem))
    for i in range(n_queries):
        idx_next_10 = np.random.choice(range(x_rem.shape[0]),size=int(x.shape[0]*.1),replace=False)
        x_rem_10 = x_rem[idx_next_10]
        y_rem_10 = y_rem[idx_next_10]
        
        for j in range(x_rem_10.shape[0]):
            query_idx, query_instance = committee.query(x_rem_10)
            committee.teach(x_rem_10[query_idx],y_rem_10[query_idx])
            x_rem_10 = np.delete(x_rem_10,query_idx,0)
            y_rem_10 = np.delete(y_rem_10,query_idx,0)
            
        x_rem = np.delete(x_rem, idx_next_10, 0) 
        y_rem = np.delete(y_rem, idx_next_10, 0) 
        comm_perform.append(committee.score(x_rem,y_rem))
    return comm_perform

[0.972972972972973]


# Evaluation